# Creating a curve from training data


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
URL_keys_leg = {
    "Exp2" : '1u2JN_yVS9I-aDDk-xtRFLs9SDdrUNo0f',
#    "Exp3" : '1qolBHxLSiGi-bmDUZ-V0WzaCay7UaA3x',
    "Exp4(5)" : '1eEh3oZpFoCo4dGdcVoGH6kuwIe7QY6rr',
    "Exp5" : '1P3E_T65CdJncewSVA-3H8hveejoqcdy4',
#    "Exp6(7)" : '1kFniM6vHQVy659t0I4R86xH6bfGgo97K',
    "Exp7(8)" : '19j6veOIvJoe3aln1MURMChglvMA2F5AS',
    "Exp8(9)" : '1akmRWg8bhkKlAQV0XimEPTzsa7_UvazU',
#    "Exp9(10)" : '1llYUn_deIMOUQl14-aJL9Gw5BqT3nH53',
    "Exp10(11)" : '1hdnEeKw7eb2bEvJLbtd12irF3N5qIFmc'
}

classification = {
    "Exp2" : "good",
    "Exp3" : "bad",
    "Exp4(5)" : "good",
    "Exp5" : "good",
    "Exp6(7)" : "bad",
    "Exp7(8)" : "good",
    "Exp8(9)" : "good",
    "Exp9(10)" : "bad",
    "Exp10(11)" : "good"
}

In [3]:
def import_from_google (ID,name):
    google_sheet_ID=ID
    work_sheet=name
    path='https://docs.google.com/spreadsheets/d/{0}/gviz/tq?tqx=out:csv&sheet={1}'.format(
        google_sheet_ID,
        work_sheet
    )

    data = pd.read_csv(path)
    
    if name=="Gyroscope":
        data.columns = [c.replace(' ', '_') for c in data.columns]
        data.columns = [c.replace('_(rad/s)', '') for c in data.columns]
        data.columns = [c.replace('_(s)', '') for c in data.columns]
    else:
        data.columns = [c.replace(' ', '_') for c in data.columns]
        data.columns = [c.replace('_(m/s^2)', '') for c in data.columns]
        data.columns = [c.replace('_(s)', '') for c in data.columns]
    
    return data

def print_graph (prefix,title,graphdata):
    plt.figure(figsize=(15,5))
    if title=="Gyroscope":
        title=prefix+" "+title
        plt.title(title)
#        plt.ylim(-6,6)
        plt.plot(graphdata.index, graphdata.Gyroscope_x, color="red")
        plt.plot(graphdata.index, graphdata.Gyroscope_y, color="blue")
        plt.plot(graphdata.index, graphdata.Gyroscope_z, color="green")
        plt.show()
        
    if title=="Accelerometer":
        title=prefix+" "+title
        plt.title(title)
        plt.plot(graphdata.index, graphdata.Acceleration_x, color="red")
        plt.plot(graphdata.index, graphdata.Acceleration_y, color="blue")
        plt.plot(graphdata.index, graphdata.Acceleration_z, color="green")
        plt.show()
        
def limit (data,idx_max,idx_min,delta):
        return data.loc[idx_max_acc-delta:idx_max_acc+delta,:] #omezení rozsahu na +/- deltu

In [5]:
delta=100
n=0

for i in URL_keys_leg:
    data_gyr = import_from_google(URL_keys_leg[i],"Gyroscope")
    data_acc = import_from_google(URL_keys_leg[i],"Accelerometer")

#    data_acc.describe()
    
    idx_max_acc=data_acc["Acceleration_y"].idxmax() #výpočet indexu maxima
    idx_min_acc=data_acc["Acceleration_y"].idxmin() #výpočet indexu maxima

    delta_time=data_acc.loc[idx_min_acc,"Time"]-data_acc.loc[idx_max_acc,"Time"] #čas maximum - minimum
    
    graphdata_gyr=limit(data_gyr,idx_max_acc,idx_min_acc,delta)
    graphdata_gyr=graphdata_gyr.reset_index(drop=True) #reset indexu
    graphdata_acc=limit(data_gyr,idx_max_gyr,idx_min_acc,delta) # <--------TADY NĚKDE JE CHYBA ----------
    graphdata_acc=graphdata_acc.reset_index(drop=True) #reset indexu

        
    if i=="Exp2":
        graphdata_gyr=limit(data_gyr,idx_max_acc,idx_min_acc,delta)
        graphdata_acc=limit(data_acc,idx_max_acc,idx_min_acc,delta)
      
        final_gyr=graphdata_gyr.drop(columns="Time")
        final_acc=graphdata_acc.drop(columns="Time")
    
    final_gyr=final_gyr+graphdata_gyr.drop(columns="Time")
    final_acc=final_acc+graphdata_acc.drop(columns="Time")
    
        
    print("-------------- Experiment ",i,"-------------")
    print("quality=",classification[i])
    print(final_gyr.describe())
    print(final_acc.describe())    
#    print("idx max=",idx_max_acc)
#    print("idx min=",idx_min_acc)
#    print("delta time=",delta_time)

#    print(graphdata_gyr.head(n=10))
#    print(graphdata_acc.head(n=10))
#    print("---FINAL---")
#    print(final_gyr.head(n=10))
#    print(final_acc.head(n=10))
    
    print_graph(i,"Gyroscope",graphdata_gyr)
    print_graph(i,"Accelerometer",graphdata_acc)
    
    n=n+1

    
#--- final ---    

final_gyr=final_gyr/n #normalizace výsledné křivky
final_acc=final_acc/n #normalizace výsledné křivky

print("---FINAL-FINAL---")
print(final_gyr.describe())
print(final_acc.describe())

#print(final_gyr.head(n=10))    
#print(final_acc.head(n=10))

print_graph("Final","Gyroscope",final_gyr)
print_graph("Final","Accelerometer",final_acc)

NameError: name 'idx_max_gyr' is not defined